In [2]:
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  InputLayer
from tensorflow.keras.layers import  Dense

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-03-09 14:28:42.120970: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
#Load the data
df = pd.read_json('transactions.txt', lines=True)
df.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,,0.0,,,,False,,,False,False
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,,0.0,,,,True,,,False,False
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,,0.0,,,,False,,,False,False
3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,...,,0.0,,,,False,,,False,False
4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,...,,0.0,,,,True,,,False,False


In [14]:
#print columns and their respective types
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786363 entries, 0 to 786362
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   accountNumber             786363 non-null  int64  
 1   customerId                786363 non-null  int64  
 2   creditLimit               786363 non-null  int64  
 3   availableMoney            786363 non-null  float64
 4   transactionDateTime       786363 non-null  object 
 5   transactionAmount         786363 non-null  float64
 6   merchantName              786363 non-null  object 
 7   acqCountry                786363 non-null  object 
 8   merchantCountryCode       786363 non-null  object 
 9   posEntryMode              786363 non-null  object 
 10  posConditionCode          786363 non-null  object 
 11  merchantCategoryCode      786363 non-null  object 
 12  currentExpDate            786363 non-null  object 
 13  accountOpenDate           786363 non-null  o

In [16]:
print(Counter(df["isFraud"]))

Counter({False: 773946, True: 12417})


In [18]:
#Do not include: accountNumber and customerId these columns are the same (index)
features = df.iloc[:,2:28] 
features.head()

,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,...,transactionType,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch
0,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,01,rideshare,...,PURCHASE,,0.0,,,,False,,,False
1,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,01,entertainment,...,PURCHASE,,0.0,,,,True,,,False
2,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,01,mobileapps,...,PURCHASE,,0.0,,,,False,,,False
3,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,01,mobileapps,...,PURCHASE,,0.0,,,,False,,,False
4,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,01,fastfood,...,PURCHASE,,0.0,,,,True,,,False


In [19]:
labels = df['isFraud']
labels.head()

0    False
1    False
2    False
3    False
4    False
Name: isFraud, dtype: bool

In [20]:
#print the number of features in the dataset
print("Number of features/columns: ", features.shape[1]) 
#print the number of samples in the dataset
print("Number of samples/rows: ", features.shape[0]) 
#see useful summary statistics for numeric features
print(features.describe()) 

Number of features/columns:  26
Number of samples/rows:  786363
         creditLimit  availableMoney  transactionAmount        cardCVV  \
count  786363.000000   786363.000000      786363.000000  786363.000000   
mean    10759.464459     6250.725369         136.985791     544.467338   
std     11636.174890     8880.783989         147.725569     261.524220   
min       250.000000    -1005.630000           0.000000     100.000000   
25%      5000.000000     1077.420000          33.650000     310.000000   
50%      7500.000000     3184.860000          87.900000     535.000000   
75%     15000.000000     7500.000000         191.480000     785.000000   
max     50000.000000    50000.000000        2011.540000     998.000000   

          enteredCVV  cardLast4Digits  currentBalance  
count  786363.000000    786363.000000   786363.000000  
mean      544.183857      4757.417799     4508.739089  
std       261.551254      2996.583810     6457.442068  
min         0.000000         0.000000        

In [21]:
print("Number for rows in the predictor variable is", labels.shape[0])
labels.describe()

Number for rows in the predictor variable is 786363


count     786363
unique         2
top        False
freq      773946
Name: isFraud, dtype: object

In [ ]:
#Split the data
#one-hot encoding for categorical variables
features = pd.get_dummies(features) 
#split the data into training and test data
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42) 

In [ ]:
#Split the data
#one-hot encoding for categorical variables
features = pd.get_dummies(features) 
#split the data into training and test data
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42) 

In [ ]:
#design the model
model = Sequential()
model.add(InputLayer(input_shape=(data_train.shape[1],))) #input layer
model.add(Dense(28, activation='relu')) #hidden layer
model.add(Dense(2, activation='sigmoid')) #the output layer is a sigmoid with 2 units (0 or 1)

In [ ]:
# Compile the model.
model.compile(optimizer="adam",
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.Precision(),
                      ])

In [ ]:
model.fit(
    X_train_scaled,
    y_train,
    validation_split=.215,
    epochs=100,
    verbose=1,
)

In [ ]:
hist = pd.DataFrame(model.history.history)

In [ ]:
hist.head()